In [1]:
import os
import sys

sys.path.insert(0, '../src')

from decimal import Decimal
from datetime import datetime

from the_pitch.data import Query
from the_pitch.simulators import PitchSimulator, Scorer
from the_pitch.strategies import StrategyFactory

In [2]:
symbols=['DAL', 'AAL', 'ALK', 'BA']

if not os.path.exists('./data/airlines.csv'):
    df = Query.get_pricing_data(source='yahoo', symbols=symbols, start='2017-01-01', end='2021-01-01')
    df.to_csv('./data/airlines.csv')

dataset = Query.load_dataset(csv = './data/airlines.csv', now='2018-06-01')

In [3]:
simulator = PitchSimulator(dataset)

In [4]:
## get strategy, indicators
iterations = [
    StrategyFactory.create_sma_cross_price(
        id='TS1',
        symbols=symbols,
        period1=20,
    ),
    StrategyFactory.create_price_revert_to_sma(
        id='TS2',
        symbols=symbols,
        period1=20,
        pl=Decimal('.12'),
        pm=Decimal('.05')
    ),
    StrategyFactory.create_sma_cross_sma(
        id='TS3',
        symbols=symbols,
        period1=50,
        period2=200
    ),
]

runs = []
for strategy, indicators in iterations:

    ## run simulator
    operations = simulator.run(
        indicators=indicators,
        strategies=[ strategy ]
    )

    runs.append(operations)

In [5]:
for run in runs:
    scores = Scorer.score(run)
    print(scores)

{'TS1': {'DAL': Decimal('18.10'), 'AAL': Decimal('-8.58'), 'ALK': Decimal('11.40'), 'BA': Decimal('-144.48')}}
{'TS2': {'AAL': Decimal('-21.14'), 'DAL': Decimal('19.75'), 'ALK': Decimal('25.40'), 'BA': Decimal('-52.75')}}
{'TS3': {'DAL': Decimal('-9.95'), 'ALK': Decimal('-2.18'), 'BA': Decimal('-52.38'), 'AAL': Decimal('0.0')}}


In [6]:
## expected,
{
    'TS1': {
        'DAL': Decimal('-9.95'),
        'ALK': Decimal('-2.18'),
        'BA': Decimal('-52.38'),
        'AAL': Decimal('0.0')
    }
}

{'TS1': {'DAL': Decimal('-22.11'),
  'ALK': Decimal('-2.18'),
  'BA': Decimal('-52.38'),
  'AAL': Decimal('0.0')}}